In [1]:
import pandas as pd
import holidays

#!pip install holidays
#!pip install openrouteservice

In [2]:
TRAFFIC_DATA_PATH = [
    'traffic_flow_2018.csv',
    'traffic_flow_2019.csv'
]

WEATHER_DATA_PATH = [
    'weather_observed_2018.csv',
    'weather_observed_2019.csv'
]


AIR_QUALITY_PATH = [
    'air_quality_2018.csv',
    'air_quality_2019.csv'
]

# df = pd.read_csv('../data/traffic_flow_2018.csv', encoding='latin-1')

In [3]:
pd.set_option("display.max_colwidth", 500)

In [4]:
pt_holidays = holidays.CountryHoliday('Portugal')

In [5]:
def load_data(data_paths, date_col, value_cols, agg_operation = 'sum'):
    concat_data = []

    for file in data_paths:
        print("Reading file ", file)
        df = pd.read_csv('../data/' + file, encoding='latin-1')
        
        # print(df.head())

        df[date_col] = pd.to_datetime(df[date_col])
        df_resampled = df.set_index(date_col).groupby('entity_id')[value_cols].resample('H').agg(agg_operation).reset_index()
        concat_data.append(df_resampled)

    return pd.concat(concat_data)

In [6]:
"""
import numpy as np
import requests
tmp = requests.get('https://wdl-data.fra1.digitaloceanspaces.com/porto-digital/POIs.json').json()
ret = []

for count, point_of_interest in enumerate(tmp['points_of_interest']):
    category = point_of_interest.get('category')[0]['value']
    location = point_of_interest.get('location')
    created = point_of_interest.get('created')
    
    if 'point' in location:
        location = location.get('point')[0].get('Point').get('posList')
    else:
        location = np.nan
    description = point_of_interest.get('description')[0]['value']
    
    ret.append({
        'id': count,
        'created_at': created,
        'category': category,
        'location': location,
        'description': description
    })

df_pois = pd.DataFrame.from_records(ret)
df_pois.to_csv('../data/pois_processed.csv')
"""

"\nimport numpy as np\nimport requests\ntmp = requests.get('https://wdl-data.fra1.digitaloceanspaces.com/porto-digital/POIs.json').json()\nret = []\n\nfor count, point_of_interest in enumerate(tmp['points_of_interest']):\n    category = point_of_interest.get('category')[0]['value']\n    location = point_of_interest.get('location')\n    created = point_of_interest.get('created')\n    \n    if 'point' in location:\n        location = location.get('point')[0].get('Point').get('posList')\n    else:\n        location = np.nan\n    description = point_of_interest.get('description')[0]['value']\n    \n    ret.append({\n        'id': count,\n        'created_at': created,\n        'category': category,\n        'location': location,\n        'description': description\n    })\n\ndf_pois = pd.DataFrame.from_records(ret)\ndf_pois.to_csv('../data/pois_processed.csv')\n"

In [7]:
df_pois = pd.read_csv('../data/pois_processed.csv')

In [8]:
df_pois.head()

,Unnamed: 0,id,created_at,category,location,description
0,0,0,2019-05-06T14:57:05.466000Z,Restaurantes,41.15743895 -8.62750399,"O Restaurante Churrascão Gaucho encontra-se encerrado, de forma a prevenir o surto do Covid-19.\n\nUm dos mais antigos restaurantes de gastronomia brasileira no Porto.\nPossui uma decoração rústica e elegante, num ambiente sofisticado."
1,1,1,2019-05-06T14:57:05.473000Z,Restaurantes,41.14073900 -8.61242000,"O Restaurante da Alzira, fundado em 1984, situa-se na zona da Ribeira do Porto, onde outrora eram os armazéns de bacalhau, num espaço arquitectónico compreendido entre os séculos XVII e XIX que foi ampliado e restaurado segundo o projecto do arquitecto Miguel Guedes. Serve comida tradicional tendo como especialidades o polvo, o bacalhau e o cabrito assados no forno. Apresenta também uma boa variedade de saladas e menu vegetariano."
2,2,2,2019-05-06T14:57:05.471000Z,Restaurantes,41.14580771 -8.60511089,"O restaurante Irmãos Unidos encontra-se encerrado, de forma a prevenir o surto do Covid-19.\n\nRestaurante localizado próximo da Praça dos Poveiros, com uma ementa baseada na cozinha tradicional portuguesa, dando destaque aos pratos típicos do Porto como as Tripas à Moda do Porto, o cabrito e o bacalhau.\n\n"
3,3,3,2019-05-06T14:57:05.476000Z,Restaurantes,41.16256626 -8.59424829,"Localizado na zona das Antas, possui uma decoração rústica e um ambiente formal. Cozinha de cariz tradicional português.\r\nRestaurante “Embaixador do Vinho do Porto”, com selecção de Vinho do Porto para acompanhar a sua refeição.\r\n"
4,4,4,2019-05-06T14:57:05.476000Z,Restaurantes,41.14802700 -8.60780700,"Restaurante com largos anos de tradição, situado no centro da cidade. Dois grandes pisos, com salas confortáveis, paredes em pedra, lampiões à moda antiga, azulejos e mobiliário em madeira.\nEmenta tipicamente portuense, onde não faltam as famosa tripas e o bacalhau.\nRestaurante “Embaixador” do Vinho do Porto, com selecção de vinho do Porto para acompanhar a sua refeição"


In [9]:
data_entities = pd.read_csv('../data/data_entities.csv', sep=',', encoding='latin-1', error_bad_lines=False)

In [10]:
# Using tabular data, there's no real way to add the features.
# We'll add the closest weather/noise sensor to each traffic sensor, as we want to predict traffic. 

In [11]:
traffic_flow_entities = data_entities[data_entities.entity_type == 'TrafficFlowObserved']
weather_entities = data_entities[data_entities.entity_type == 'WeatherObserved']

In [12]:
!pip install geopy

In [13]:
from geopy.distance import geodesic

#centroid = (data_entities['latitude'].astype(float).mean(), data_entities['longitude'].astype(float).mean())
#data_entities['distance_to_centroid'] = data_entities.apply(lambda x: geodesic((x['latitude'], x['longitude']), centroid).meters, axis=1)

In [15]:
traffic_flow_entities

,entity_id,name,latitude,longitude,entity_type,street,start_date,end_date
0,testsixsq,Rua dos Campeões Europeus,41.162590,-8.585198,TrafficFlowObserved,Rua dos Campeões Europeus,NaN,NaN
1,urn:ngsi-ld:TrafficFlowObserved:porto:traffic:CMP:CT10Z2,R. Fernandes Tomás - 24 de Agosto,41.148406,-8.599094,TrafficFlowObserved,Rua de Fernandes Tomás,NaN,NaN
2,urn:ngsi-ld:TrafficFlowObserved:porto:traffic:CMP:CT10Z3,R. Camões - N-S,41.154960,-8.610091,TrafficFlowObserved,Rua de Camões,NaN,NaN
3,urn:ngsi-ld:TrafficFlowObserved:porto:traffic:CMP:CT10Z4,Av. Boavista - Casa da Música - E-O,41.158701,-8.633546,TrafficFlowObserved,Avenida da Boavista,NaN,NaN
4,urn:ngsi-ld:TrafficFlowObserved:porto:traffic:CMP:CT10Z5,Via Panorâmica - Faixa BUS,41.151096,-8.633818,TrafficFlowObserved,Rua de Entre Campos,NaN,NaN
...,...,...,...,...,...,...,...,...
125,urn:ngsi-ld:TrafficFlowObserved:porto:traffic:VolumLight:0_4,Av. Sidónio Pais - Câmara,41.164327,-8.636619,TrafficFlowObserved,Avenida Sidónio Pais,NaN,NaN
126,urn:ngsi-ld:TrafficFlowObserved:porto:traffic:VolumLight:0_5,Av. Sidónio Pais - Câmara,41.164327,-8.636619,TrafficFlowObserved,Avenida Sidónio Pais,NaN,NaN
127,urn:ngsi-ld:TrafficFlowObserved:porto:traffic:VolumLight:0_6,Av. Sidónio Pais - Câmara,41.164327,-8.636619,TrafficFlowObserved,Avenida Sidónio Pais,NaN,NaN
178,urn:ngsi-ld:TrafficFlowObserved:porto:traffic:CMP:CT7Z10,Av.Boavista O-E - Parque da Cidade,41.165238,-8.673290,TrafficFlowObserved,NaN,NaN,NaN


In [16]:
import numpy as np

def return_closest_weather_entity(weather_entities, traffic_entity):
    # Return closest weather entity for a given traffic_entity

    distances = []
    for i, weather_entity in weather_entities.reset_index().iterrows():
        coord_traffic = traffic_entity['latitude'], traffic_entity['longitude']
        coord_weather = weather_entity['latitude'], weather_entity['longitude']

        distances.append(geodesic(coord_traffic, coord_weather).meters)

    closest_weather_sensor = weather_entities.iloc[np.argmin(distances)][['entity_id']].values
    
    return closest_weather_sensor

mapping_weather_entities_traffic = {}

for i, traffic_entity in traffic_flow_entities.reset_index().iterrows():
    
    mapping_weather_entities_traffic[traffic_entity['entity_id']] = return_closest_weather_entity(weather_entities, traffic_entity)[0]

In [19]:
date_col = 'dateobservedfrom'
value_cols = ['intensity']
files_name = TRAFFIC_DATA_PATH

traffic_data = load_data(files_name, date_col, value_cols, agg_operation = 'sum')
traffic_data.rename(columns={'dateobservedfrom': 'dateobserved'}, inplace=True)

WEATHER_COLS = [
    'barometricpressure',
    'dewpoint',
    'precipitation',
    'relativehumidity',
    'solarradiation',
    'temperature',
    'winddirection',
    'windspeed',
    'illuminance',
    'uvindexmax',
    'pressuretendency'
]

weather_data = load_data(data_paths = WEATHER_DATA_PATH, date_col = 'dateobserved', value_cols = WEATHER_COLS, agg_operation = 'mean')

AIR_QUALITY_COLS = [
    'no2',
    'o3',
    'ox',
    'co',
    'pm1',
    'pm25',
    'pm10'
]

air_quality_data = load_data(data_paths = AIR_QUALITY_PATH, date_col = 'dateobserved', value_cols = AIR_QUALITY_COLS, agg_operation = 'mean')


Reading file  traffic_flow_2018.csv
Reading file  traffic_flow_2019.csv


C:\Users\paulo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Reading file  weather_observed_2018.csv


C:\Users\paulo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Reading file  weather_observed_2019.csv
Reading file  air_quality_2018.csv
Reading file  air_quality_2019.csv


In [21]:
weather_data.head()

,entity_id,dateobserved,barometricpressure,dewpoint,precipitation,relativehumidity,solarradiation,temperature,winddirection,windspeed,illuminance,uvindexmax,pressuretendency
0,urn:ngsi-ld:AirQualityObserved:porto:weather:ubiwhere:5ad613dd3da2520542bc871a,2018-06-29 18:00:00,1011.000000,NaN,0.0,0.833000,39.099998,19.700001,312.000000,1.500000,NaN,NaN,NaN
1,urn:ngsi-ld:AirQualityObserved:porto:weather:ubiwhere:5ad613dd3da2520542bc871a,2018-06-29 19:00:00,1010.933339,NaN,0.0,0.856417,13.758334,19.533334,269.750000,1.236111,NaN,NaN,NaN
2,urn:ngsi-ld:AirQualityObserved:porto:weather:ubiwhere:5ad613dd3da2520542bc871a,2018-06-29 20:00:00,1011.224991,NaN,0.0,0.887917,1.291667,18.875000,283.000000,1.814815,NaN,NaN,NaN
3,urn:ngsi-ld:AirQualityObserved:porto:weather:ubiwhere:5ad613dd3da2520542bc871a,2018-06-29 21:00:00,1011.225001,NaN,0.0,0.924417,0.508333,18.400000,286.916667,1.046296,NaN,NaN,NaN
4,urn:ngsi-ld:AirQualityObserved:porto:weather:ubiwhere:5ad613dd3da2520542bc871a,2018-06-29 22:00:00,1011.108327,NaN,0.0,0.913167,0.500000,18.708333,248.500000,0.553241,NaN,NaN,NaN


In [22]:
traffic_data['closest_weather_entity_id'] = traffic_data['entity_id'].map(mapping_weather_entities_traffic)

In [ ]:
air_quality_data.head()

In [ ]:
data_entities.head()

In [ ]:
data_entities['entity_type'].value_counts()

In [ ]:
from geopy.distance import geodesic

centroid = (data_entities['latitude'].astype(float).mean(), data_entities['longitude'].astype(float).mean())
data_entities['distance_to_centroid'] = data_entities.apply(lambda x: geodesic((x['latitude'], x['longitude']), centroid).meters, axis=1)

## Sensor Location

There seem to be a total of 130 traffic flow sensors, 27 weather sensors, 18 noise sensors and 18 air quality sensors. 

The map below determines their distribution throughout the city of Porto.

- **Traffic Sensors**: In the area of "Foz" and "Campanhã", there seem to be less traffic sensors compared to the remaining major areas of Porto: Boavista, Antas, Baixa (Downtown) and Antas. 
- **Noise Sensors** - They're very close to the traffic sensors, so shouldn't really give any complementary information
- **Air Quality Sensors** - Three are in Viseu, not Porto. The remaining seem sparse and distant from traffic sensors, so it's hard to correlate a specific traffic sensor's intensity with the air quality in a different region (something that might also depend on the wind direction and intensity at that time). Their use for this problem could be in measuring the correlation between high traffic days and air quality a few moments later.
- **Weather Sensors** - There seems to be a relevant amount of coverage throughout the city 

In [ ]:
import folium

def plot_sensors_location(data_entities):

    m = folium.Map(location=[41.16181, -8.62057], zoom_start=14)

    for indice, row in data_entities.loc[data_entities["entity_type"]=="TrafficFlowObserved"].iterrows():
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=row['name'],
            icon=folium.Icon(color="red", icon='automobile', prefix='fa')
            ).add_to(m)

    for indice, row in data_entities.loc[data_entities["entity_type"]=="NoiseLevelObserved"].iterrows():
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=row['name'],
            icon=folium.Icon(color="green", icon='glyphicon glyphicon-bullhorn')
            ).add_to(m)

    for indice, row in data_entities.loc[data_entities["entity_type"]=="AirQualityObserved"].iterrows():
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=row['name'],
            icon=folium.Icon(color="blue", icon='leaf', prefix='fa')
            ).add_to(m)

    for indice, row in data_entities.loc[data_entities["entity_type"]=="WeatherObserved"].iterrows():
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=row['name'],
            icon=folium.Icon(color="orange", icon='sun-o', prefix='fa')
            ).add_to(m)
    return m

m = plot_sensors_location(data_entities)

m

In [ ]:
df_resampled = traffic_data.merge(data_entities, on='entity_id')
#df_resampled = df_resampled.merge(weather_data, on=['entity_id', 'dateobserved'])
#df_resampled = df_resampled.merge(traffic_data, on=['entity_id', 'dateobserved'])

In [ ]:
df_resampled.latitude = df_resampled.latitude.astype(float)
df_resampled.longitude = df_resampled.longitude.astype(float)

In [ ]:
#latbins = pd.cut(df_resampled.latitude, 8, labels=[f'lat{k}' for k in range(1,8+1)], retbins=True)
#df_resampled = pd.concat([df_resampled, pd.get_dummies(latbins[0])], axis=1)
#longbins = pd.cut(df_resampled.longitude, 8, labels=[f'long{k}' for k in range(1,8+1)], retbins=True)
#df_resampled = pd.concat([df_resampled, pd.get_dummies(longbins[0])], axis=1)

In [ ]:
df_resampled.head()

In [ ]:
df_resampled = df_resampled.sort_values(by=['entity_id', 'dateobserved'], ascending=True)

## Feature Extraction

In [ ]:
import numpy as np

def get_date_features(df_resampled, date_col, suffix):
    df_resampled[f'day_{suffix}'] = df_resampled[date_col].dt.day
    df_resampled[f'hour_{suffix}'] = df_resampled[date_col].dt.hour
    df_resampled[f'month_{suffix}'] = df_resampled[date_col].dt.month
    df_resampled[f'dayofweek_{suffix}'] = df_resampled[date_col].dt.dayofweek
    df_resampled[f'year_{suffix}'] = df_resampled[date_col].dt.year
    # df_resampled[f'quarter_{suffix}'] = df_resampled[date_col].dt.q
    df_resampled[f'is_holiday_{suffix}'] = df_resampled[date_col].apply(lambda x: x in pt_holidays)
    df_resampled[f'is_year_end_{suffix}'] = df_resampled[date_col].dt.is_year_end
    df_resampled[f'is_weekend_{suffix}'] = np.where(df_resampled[f'dayofweek_{suffix}'].isin([5, 6]), 1, 0)
                                                  
    return df_resampled

In [ ]:
df_resampled = get_date_features(df_resampled, 'dateobserved', 'today')

In [ ]:
# Set target

df_resampled['target'] = df_resampled.groupby('entity_id')['intensity'].shift(-24)
df_resampled['dateobserved_target'] = df_resampled['dateobserved'] + pd.DateOffset(hours=24)

In [ ]:
df_resampled = get_date_features(df_resampled, 'dateobserved_target', 'tomorrow')

In [ ]:
df_resampled.head()

Here's an example of what the target looks like.
For this entity, on 2018-10-07 00:00, we observe an intensity of 104. The target, the number of cars on the following day at 00:00, is 95, which appears on the following row.

In [ ]:
df_resampled[
    (
        (df_resampled.dateobserved == pd.to_datetime('2018-10-07 00:00:00')) | (df_resampled.dateobserved == pd.to_datetime('2018-10-08 00:00:00'))
    ) & (df_resampled.entity_id == 'urn:ngsi-ld:TrafficFlowObserved:porto:traffic:CMP:CT10Z2')
][['entity_id', 'dateobserved', 'intensity', 'target']]

In [ ]:
# Feature Engineering

df_resampled['yesterday_value'] = df_resampled.groupby('entity_id')['intensity'].shift(24)

In [ ]:
df_resampled.sort_values(by='intensity', ascending=False)

In [ ]:
df_resampled.columns

In [ ]:
def rolling_fts_intensity(df_resampled, group_cols, window, agg_cols = ['mean', 'max', 'min', 'std']):
    rolling_fts = df_resampled.groupby(group_cols)['intensity'].shift(1).rolling(window).agg(['mean', 'max', 'min', 'std'])
    
    group_cols_str = '_'.join(group_cols)
    rolling_fts.columns = [
        f'intensity_{group_cols_str}_{str(window)}_mean',
        f'intensity_{group_cols_str}_{str(window)}_max',
        f'intensity_{group_cols_str}_{str(window)}_min',
        f'intensity_{group_cols_str}_{str(window)}_std',
    ]
    df_resampled = pd.concat([df_resampled, rolling_fts], axis=1)
    return df_resampled

In [ ]:
for day in [24, 24*2, 24*3, 24*4, 24*5, 24*6]:
    df_resampled = rolling_fts_intensity(df_resampled, ['entity_id'], day)

In [ ]:
df_resampled.tail()

In [ ]:
df_resampled = rolling_fts_intensity(df_resampled, ['entity_id', 'dayofweek_today'], 24)

In [ ]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

In [ ]:
df_resampled.columns

In [ ]:
COLS_TO_REMOVE = [
    'entity_id',
    'dateobserved',
    'name',
    'latitude',
    'longitude',
    'entity_type',
    'street',
    'start_date',
    'end_date',
    'target',
    'dateobserved_target'
]

COLS_TO_KEEP = [x for x in df_resampled.columns if x not in COLS_TO_REMOVE]

In [ ]:
COLS_TO_KEEP

In [ ]:
df_resampled.columns

In [ ]:
base_sensor_name = "urn:ngsi-ld:TrafficFlowObserved:porto:traffic:CMP:"
outlier_sensor_list = [
    "CT10Z2",
    "CT10Z3",
    "CT11Z2",
    "CT11Z8",
    "CT12Z3",
    "CT13Z9",
    "CT14Z2",
    "CT14Z9",
    "CT15Z2",
    "CT15Z3",
    "CT17Z2",
    "CT19Z3",
    "CT1Z10",
    "CT1Z2",
    "CT1Z3",
    "CT1Z4",
    "CT1Z6",
    "CT1Z7",
    "CT1Z9",
    "CT20Z1",
    "CT21Z1",
    "CT2Z2",
    "CT3Z5",
    "CT4Z4",
    "CT5Z8",
    "CT7Z5",
    "CT7Z9",
    "CT8Z2",
    "CT8Z5",
    "CT8Z8",
    "CT9Z2",
    "CT9Z3"
]
outlier_sensor_list = [base_sensor_name+n for n in outlier_sensor_list]
outlier_sensor_list.append("testsixsq")

df_resampled = df_resampled[
    ~df_resampled.entity_id.isin(outlier_sensor_list)
]

In [ ]:
# Train baseline model

# COLS = ['intensity', 'day', 'hour', 'month', 'dayofweek', 'year', 'quarter', 'is_year_end', 'is_holiday', 'yesterday_value']
#COLS = COLS + [f'lat{k}' for k in range(1,8+1)]
# COLS = COLS + [f'long{k}' for k in range(1,8+1)]

X_train = df_resampled[0:int(0.8*len(df_resampled))]
X_train = X_train[~X_train.target.isna()]

X_test = df_resampled[int(0.8*len(df_resampled)):]
X_test = X_test[~X_test.target.isna()]

In [ ]:
import numpy as np

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

#y_train = (X_train['target'] - X_train['intensity'])/(X_train['intensity'] + 0.001)
y_train = X_train['target']

# y_train = X_train['target']
#y_train = scaler.fit_transform(y_train.values.reshape(-1, 1))

In [ ]:
xgb = XGBRegressor(n_estimators=100)

xgb.fit(X_train[COLS_TO_KEEP].fillna(9999).astype(float), y_train)

In [ ]:
y_pred = xgb.predict(X_test[COLS_TO_KEEP].fillna(9999).astype(float))

In [ ]:
y_pred

In [ ]:
X_test['pred_score'] = y_pred
#X_test['target'] = (X_test['target'] - X_test['intensity'])/(X_test['intensity'] + 0.001)
#X_test['target'] = X_test['target'].fillna(0)

In [ ]:
# Predicting that today's intensity will be the same as the target for baseline

dummy_regressor = X_test['intensity']

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr, pearsonr

print("MAE", mean_absolute_error(y_pred, X_test['target']))
print("MSE", mean_squared_error(y_pred, X_test['target']))
print("Spearman Rank", spearmanr(y_pred, X_test['target']))
print("Pearson Rank", pearsonr(y_pred, X_test['target']))

In [ ]:
print("MAE", mean_absolute_error(dummy_regressor, X_test['target']))
print("MSE", mean_squared_error(dummy_regressor, X_test['target']))
print("Spearman Rank", spearmanr(dummy_regressor, X_test['target']))
print("Pearson Rank", pearsonr(dummy_regressor, X_test['target']))

In [ ]:
X_test[X_test['intensity'] > 6000].head()[['dateobserved', 'entity_id', 'intensity', 'dayofweek_today', 'pred_score', 'target']]

In [ ]:
X_test[(X_test['dateobserved'] == '2019-08-29 12:00:00') & (X_test['entity_id'] == 'urn:ngsi-ld:TrafficFlowObserved:porto:traffic:CMP:CT7Z3')]

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(y_pred, X_test['target'], alpha=0.2)

In [ ]:
xgb.get_booster().get_score(importance_type='weight')

In [ ]:
!pip install shap

In [ ]:
import shap

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
explainer = shap.Explainer(xgb)
shap_values = explainer(X_test[COLS_TO_KEEP].fillna(9999).astype(float))

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
tmp = X_test[X_test.entity_id == 'urn:ngsi-ld:TrafficFlowObserved:porto:traffic:CMP:CT7Z3']

In [ ]:
tmp = tmp[
    (tmp.dateobserved < '2018-08-12')# & (tmp.dateobserved > '2018-05-18')
]

In [ ]:
plt.plot(tmp['dateobserved'][600:700], tmp['pred_score'][600:700], label='pred')
plt.plot(tmp['dateobserved'][600:700], tmp['target'][600:700], label='true')
plt.legend()